# Train MOT17 dataset

Train MOt17 in SageMaker

### Set up

#### 1. Set  up  accounts and role

In [1]:
import sagemaker
import boto3
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tempfile

sys.path.append('./src')


sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)

dataset_url = "https://motchallenge.net/data/MOT17.zip"

In [2]:
tmp_dir = tempfile.mkdtemp("_mot17data")

#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="object-tracking:gpu-1.1.0-201911190554"
instance_type = "ml.p3.2xlarge" 
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
commit_id =  "ba30f5811e0907e59cf199dfbd0f2ef019c3c538"

#### 3. Configure train/ test and validation datasets

Set prepare dataset to true if you want to download the 5 GB dataset from Mot 17 and update to s3. Once this is in s3 set the prepare_dataset false to avoid redoing this over and over again

In [5]:
prepare_dataset = False

In [6]:
bucket = sagemaker_session.default_bucket()
raw_bucket="aegovansagemaker"

In [7]:
s3_train="s3://{}/mot17/train/".format(bucket)


s3_val="s3://{}/mot17/val/".format(bucket)

s3_test="s3://{}/mot17/test/".format(bucket)



s3_output_path= "s3://{}/market1501_output/".format(bucket)

## Prepare dataset split

In [26]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os
from datetime import datetime

def uploadfile(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """
        #print("Uploading {} to {}".format(localpath, s3path))
        
        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    assert s3path_prefix.endswith("/")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    s3_path = s3_prefix.rstrip("/")
    input_tuples = ( (f, "{}/{}".format( s3_path, f.lstrip(local_dir.lstrip("/")))) for f in glob.glob("{}/**".format(local_dir),  recursive=True) if os.path.isfile(f) )
    list_input_tuples = list(input_tuples)
    print("{} : Uploading {} files".format(datetime.now(), len(list_input_tuples)))
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, list_input_tuples)
    print("{} : upload complete".format(datetime.now(), len(list_input_tuples)))
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size



In [9]:
from sklearn.model_selection import train_test_split
import shutil
def split_train_val(train_dir_root, dest_train, dest_val, dest_test):
    """
    Expects the root path containing MOT 17 directory structure as is. 
    """
    folders = [ (train_dir_root , d)  for d in os.listdir(train_dir_root) if os.path.isdir(os.path.join(train_dir_root, d))]
    train_list, test_list = train_test_split(    folders , test_size=0.2, random_state=42)
    train_list, val_list = train_test_split(    train_list , test_size=0.1, random_state=42)


    def partition(list_to_partition, dest_dir):
        for root, d  in list_to_partition :
            dest_path = os.path.join(dest_dir, d)
            src_path =  os.path.join(train_dir_root, d)
            print("moving {} to {} ".format(src_path, dest_path))
            shutil.move(src_path, dest_path)
            
    partition(train_list, dest_train)
    partition(val_list, dest_val)
    partition(test_list, dest_test)


    

In [27]:
import urllib.request 
from datetime import datetime
from pyunpack import Archive
import zipfile

def download_dataset(dest_file, url):
    MB = 1024 * 1024 
    read_size= MB * 5
    log_freq=30
    print("Downloading file {}....".format(dest_file))
    with  urllib.request.urlopen(url) as u:
        with open(dest_file, "wb") as o:
            total_read = 0
            i = 0
            while True:
                buffer = u.read(read_size)
                if not buffer:
                    break
                total_read += len(buffer)
                o.write(buffer)
                i += 1
                if i % log_freq == 0 : print("{} so far...{} MB ".format(datetime.now(), total_read/MB))
                

def extract_zip(zip_file, dest_path):
   
    print("Is valid zip file: {}".format(zipfile.is_zipfile(zip_file)))
      
    with zipfile.ZipFile(zip_file) as z:
        print("Invalid members...{}".format(z.testzip()))
        
    with zipfile.ZipFile(zip_file, 'r', allowZip64=True) as z: 
        print("Extracting files to {}....".format(dest_path))
        for member in z.infolist():
            try:
                z.extract(member, dest_path)
            except zipfile.error as e:
                Warning(e)
        print("Completed..")
   

In [ ]:
%%time 


if prepare_dataset:
 
    raw_dataset_zip = os.path.join(tmp_dir, "mot17.zip")
    raw_dataset_dir = os.path.join(tmp_dir, "data")
    os.makedirs(raw_dataset_dir,  exist_ok=True) 
    
    download_dataset(raw_dataset_zip, dataset_url)
    extract_zip (raw_dataset_zip, raw_dataset_dir )
    
    dest_train = os.path.join(tmp_dir, "train")
    dest_test = os.path.join(tmp_dir, "test")
    dest_val = os.path.join(tmp_dir, "val")
    
    src_raw_train_dir = os.path.join(raw_dataset_dir, "train")
    split_train_val(  src_raw_train_dir  , dest_train, dest_val, dest_test)
    upload_files(dest_train, s3_train)
    upload_files(dest_val, s3_val)
    upload_files(dest_test, s3_test)

else:
    print("Not preparing dataset as prepare_dataset is set to False")

 Is valid zip file True
Invalid members...test/MOT17-01-DPM/seqinfo.ini
Extracting files to /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/data....
Completed..
moving /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/data/train/MOT17-10-FRCNN to /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-10-FRCNN 
moving /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/data/train/MOT17-13-FRCNN to /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN 
moving /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/data/train/MOT17-10-SDP to /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-10-SDP 
moving /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/data/train/MOT17-05-FRCNN to /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN 
moving /var/folders/

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-DPM/img1/000725.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-DPM/img1/000725.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-DPM/img1/000633.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-DPM/img1/000633.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000714.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000714.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000438.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000438.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000806.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000806.jp

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-DPM/img1/000543.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-DPM/img1/000543.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000532.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000532.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000348.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000348.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000256.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000256.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-DPM/img1/000636.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-DPM/img1/000636.jp

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000167.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000167.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-DPM/img1/000822.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-DPM/img1/000822.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000075.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000075.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000811.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000811.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-09-FRCNN/img1/000086.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-09-FRCNN/img1/00008

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-DPM/img1/000733.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-DPM/img1/000733.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000261.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000261.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-DPM/img1/000548.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-DPM/img1/000548.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000353.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000353.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-05-FRCNN/img1/000537.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-05-FRCNN/img1/000537.jp

...

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000443.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000443.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000352.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000352.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000276.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000276.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000625.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000625.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000035.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000035.jpg
Uploading 

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-DPM/img1/000729.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-DPM/img1/000729.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000278.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000278.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000717.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000717.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000188.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000188.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000536.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000536.jpg
Uploading 

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000339.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000339.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000190.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000190.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000428.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000428.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-DPM/img1/000641.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-DPM/img1/000641.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000247.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000247.jp

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000527.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000527.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000632.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000632.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000450.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000450.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000613.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000613.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000070.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000070.jp

Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000161.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000161.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000724.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000724.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-SDP/img1/000285.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-SDP/img1/000285.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000433.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000433.jpg
Uploading /var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpovyg2zf6_mot17data/train/MOT17-13-FRCNN/img1/000344.jpg to s3://sagemaker-us-east-2-324346001917/mot17/train/MOT17-13-FRCNN/img1/000344.jp

## Start training

In [12]:
inputs = {
    "train" : s3_train,
#     "val" :s3_val
}

In [13]:
hyperparameters_exp1 = {
    "dataset":"Mot17DetectionFactory",
    "batchsize": "32",
    "epochs" : "1000",
    "learning_rate":.0001,
    "weight_decay":5e-5,
    "momentum":.9,
    "patience": 20,
    "log-level" : "INFO",
    "commit_id":commit_id
}

In [14]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ,{"Name": "trainVariance",
                     "Regex": "###score: train_loss_std### (\d*[.]?\d*)"}
                    ,{"Name": "ValVariance",
                     "Regex": "###score: val_loss_std### (\d*[.]?\d*)"}
                    ]

In [15]:
!git log -1| head -1
!git log -1| tail -1

commit ba30f5811e0907e59cf199dfbd0f2ef019c3c538
    Add requirements


In [16]:
hyperparameters = hyperparameters_exp1


In [17]:
git_config = {'repo': 'https://github.com/elangovana/object-tracking.git',
              'branch': 'master',
              'commit': hyperparameters['commit_id']
             }

In [18]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     entry_point='experiment_train.py',
                    source_dir = 'src',
                    dependencies =['src/datasets', 'src/evaluators', 'src/models'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_max_run = 60 * 60 * 24,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
#                     train_use_spot_instances = True,
#                     train_max_wait = 60 ,
                    base_job_name ="object-detection")

In [19]:
estimator.fit(inputs, wait=False)